In [3]:
import tensorflow as tf
import os 
from pathlib import Path
import urllib.request as request
import numpy as np
import matplotlib.pyplot as plt
from zipfile import ZipFile

In [4]:
class Config():
    def __init__(self):
        self.ROOT_DATA_DIR = r"C:\Practice\INeuron\FSDS\VGG16_demo\data"
        self.IMAGE_SIZE = (224,224)
        self.BATCH_SIZE = 32
        self.EPOCH = 10

    def create_dir(self, dir_path):
        os.makedirs(dir_path, exist_ok =True)
        print(f"{dir_path} created")

config = Config()        

In [19]:
config.create_dir(config.ROOT_DATA_DIR)

C:\Practice\INeuron\FSDS\VGG16_demo\data created


In [5]:
model = tf.keras.applications.VGG16(

    include_top = True, weights = "imagenet",classes=1000
    )

In [5]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
model.save("VGG16_full_model.h5")

In [7]:
model.layers[1].padding # Same padding means p = 1 to keep the image of same size

'same'

# Download Data

In [8]:
data_URL = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"

In [9]:
data_zip_file = "data.zip"
data_zip_path = os.path.join(config.ROOT_DATA_DIR,data_zip_file)

In [10]:
if not os.path.isfile(data_zip_file):
    print("Downloading started...")
    filename, headers = request.urlretrieve(data_URL, data_zip_path)
    print(f"filename : {filename} created with info  \n {headers}")
else:
    print(f"File already exists")    

filename : C:\Practice\INeuron\FSDS\VGG16_demo\data\data.zip created with info  
 Content-Type: application/zip
Content-Length: 47286322
Connection: close
Date: Fri, 26 May 2023 06:24:51 GMT
Last-Modified: Wed, 15 Mar 2017 18:46:00 GMT
ETag: "5f8c32a6554f6acb4d649776e7735e48"
x-amz-version-id: null
Accept-Ranges: bytes
Server: AmazonS3
X-Cache: Hit from cloudfront
Via: 1.1 6705fde62c89246795b7b0fe68d664cc.cloudfront.net (CloudFront)
X-Amz-Cf-Pop: TXL50-P3
X-Amz-Cf-Id: -khs9uqYPIzZl9NYfVcHCXEXBoKZJl8rk_uoUHmlEKCkQDOTUvADIQ==
Age: 11355




In [11]:
unzip_data_dirname = "unzip_data_dir" 
unzip_data_dir = os.path.join(config.ROOT_DATA_DIR, unzip_data_dirname)

if not os.path.exists(unzip_data_dir):
    os.makedirs(unzip_data_dir, exist_ok=True)
    with ZipFile(data_zip_path) as f:
        print(f"Extraction started...")
        f.extractall(unzip_data_dir)

else:
    print(f"data already extracted")        

data already extracted


In [12]:
main_data_dir = Path(r"C:\Practice\INeuron\FSDS\VGG16_demo\data\unzip_data_dir\hymenoptera_data\train")
main_data_dir

WindowsPath('C:/Practice/INeuron/FSDS/VGG16_demo/data/unzip_data_dir/hymenoptera_data/train')

In [15]:
datagen_kwargs = dict(rescale=1./255, validation_split = 0.20)

dataflow_kwargs = dict(target_size = config.IMAGE_SIZE, batch_size = config.BATCH_SIZE)

In [16]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)

In [18]:
valid_generator = valid_datagen.flow_from_directory(
    main_data_dir, subset="validation", shuffle=True, **dataflow_kwargs
)

do_data_augmentation =False

if do_data_augmentation:
    train_datagen  = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=40,
        horizontal_flip=True,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        **datagen_kwargs
    )
else:
    train_datagen = valid_datagen

train_generator = train_datagen.flow_from_directory(
    main_data_dir, subset = "training", shuffle=True, **dataflow_kwargs
)        



Found 48 images belonging to 2 classes.
Found 196 images belonging to 2 classes.


In [19]:
RGB_IMAGE_SIZE = (224,224,3)
vgg = tf.keras.applications.vgg16.VGG16(
    input_shape = RGB_IMAGE_SIZE,
    weights="imagenet",
    include_top=False
)

 3096576/58889256 [>.............................] - ETA: 21s